In [8]:
import glob
import numpy as np
import pandas as pd
import csv
import h5py
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer, Dense, Dropout, LSTM, GRU, Conv1D, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import concatenate, GlobalMaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, Reshape, TimeDistributed, Embedding, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adamax, Adadelta, Adagrad, Nadam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2
from keras import backend as K
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

import parse_files as p
from features import bigram_features0, bigram_features1, bigram_features2, bigram_features3, bigram_features4, bigram_features5

from numpy.random import seed
from tensorflow.python.keras.backend import set_session
from keras import regularizers

In [2]:
# class TransformerBlock(layers.Layer):
#   def __init__(self, embed_dim, num_heads, foldlabels, learning_rate=0.001):
#     super(TransformerBlock, self).__init__()

#     self.attention = layers.MultiHeadAttention(num_heads, embed_dim)
#     self.dropout1 = layers.Dropout(learning_rate)
#     self.norm1 = layers.LayerNormalization(epsilon=1e-6)

#     self.feed_forward_network = keras.Sequential([
#         Dense(512, activation='tanh'),
# 		    Dense(128, activation='tanh')
#     ])

#     self.droput2 = layers.Dropout(learning_rate)
#     self.norm2 = layers.LayerNormalization(epsilon=1e-6)
#     self.softmax = Dense(foldlabels.shape[1], activation='softmax', name='main_output', kernel_regularizer=l2(0.01))

#   def call(self, inputs):
#     batch_size, seq_len = tf.shape(inputs)[0], tf.shape(inputs)[1]

In [3]:
# clsfeature = True
clsfeature = False


rawdata = 'hmm'
# rawdata = 'pssm'

# predtype = 'Class'
predtype = 'Fold'

# dataset = 'dd'
# dataset = 'edd'
# dataset = 'tg'
dataset = 'SCOPe'
#dataset = '25_SCOPe_DDEDDTG'

In [4]:
# if dataset == 'dd':
#     sgt_df = pd.read_csv('../Datasets/dd.csv')
# elif dataset == 'edd':
#     sgt_df = pd.read_csv('../Datasets/edd.csv')
# else:
#     sgt_df = pd.read_csv('../Datasets/tg.csv')

# sgt_df['Sequence'] = sgt_df['Sequence'].str.lower()
# sgt_df.head()

# A = []
# for i in sgt_df['Sequence']:
#   a = (" ".join(i))
#   A.append(a)
# sgt_df['Sequence'] = A
# sgt_df['Sequence'].iloc[0]

# sgt_df['Sequence'] = sgt_df['Sequence'].str.split(' ')

# sgt = SGT(kappa = 10, flatten=True)
# all_sequences = []
# useless_sequences = []
# for sequence in tqdm(range(len(sgt_df['Sequence']))):
#     try:
#         encoded_sequence = sgt.fit(sgt_df['Sequence'].iloc[sequence])
#         all_sequences.append(encoded_sequence)
#     except ValueError:
#         print('Useless Sequence')
#         useless_sequences.append(sequence)

# all_sequences = np.array(all_sequences)
# all_sequences.shape

# all_sequences = np.reshape(all_sequences, (all_sequences.shape[0], 1, all_sequences.shape[1]))
# all_sequences.shape

In [7]:
np.random.seed(0)
tf.random.set_seed(0)

labels = []
hmm = []
pssm = []
seq = []
seqlen = 200
biGram_features0 = []
biGram_features1 = []
biGram_features2 = []
biGram_features3 = []
biGram_features4 = []
biGram_features5 = []


# Load all the filenames of PSSM's
filelist = glob.glob('./data/'+dataset+'/'+rawdata+'/*.txt')

# Read all the labels of the given dataset
if dataset == "SCOPe":
	label_for_seq = pd.read_csv("./astral_2_08_final.csv") # Make sure all the sequences are in uppercase
else:
	label_for_seq = p.load_labels('./data/'+dataset+'_'+predtype+'_labels.txt')

# Read all the HMM and PSSM matrices of the given dataset
for i in range(0, len(filelist)):
	# HMM data
	if(rawdata == 'hmm'):
		seq_hmm,prob_hmm,extras_hmm = p.parse_hmm(filelist[i])
		tempseq = seq_hmm.upper()
		seq.append(tempseq)
		if dataset == "SCOPe":
			labels.append(label_for_seq.loc[label_for_seq["sequence"] == tempseq]["fold"].values[0])
		else:
			labels.append(label_for_seq[seq_hmm.upper()])
		if(clsfeature):
			biGram_features0.append(((np.append((bigram_features0(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features1.append(((np.append((bigram_features1(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features2.append(((np.append((bigram_features2(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features3.append(((np.append((bigram_features3(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features4.append(((np.append((bigram_features4(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features5.append(((np.append((bigram_features5(prob_hmm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
		else:
			biGram_features0.append(bigram_features0(prob_hmm))
			biGram_features2.append(bigram_features2(prob_hmm))
			biGram_features3.append(bigram_features3(prob_hmm))
			biGram_features4.append(bigram_features4(prob_hmm))
			biGram_features1.append(bigram_features1(prob_hmm))
			biGram_features5.append(bigram_features5(prob_hmm))

		norm_hmm = prob_hmm + 0.01
		if(len(norm_hmm) < seqlen):
			for j in range(seqlen-len(norm_hmm)):
				norm_hmm = np.concatenate((norm_hmm,norm_hmm[0]*0))
		else:
			norm_hmm = norm_hmm[:seqlen]
		hmm.append(norm_hmm)

	# PSSM data
	else:  
		seq_pssm,prob_pssm,lprob_pssm,extra_pssm = p.parse_pssm(filelist[i])
		tempseq = seq_pssm.upper()
		seq.append(tempseq)
		labels.append(label_for_seq[seq_pssm.upper()])	    
		if(clsfeature):
			biGram_features0.append(((np.append((bigram_features0(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features1.append(((np.append((bigram_features1(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features2.append(((np.append((bigram_features2(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features3.append(((np.append((bigram_features3(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features4.append(((np.append((bigram_features4(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
			biGram_features5.append(((np.append((bigram_features5(prob_pssm)), (p.get_class_label(dataset, label_for_seq[tempseq])))).reshape(1, -1)))
		else:
			biGram_features0.append(bigram_features0(prob_pssm))
			biGram_features2.append(bigram_features2(prob_pssm))
			biGram_features3.append(bigram_features3(prob_pssm))
			biGram_features4.append(bigram_features4(prob_pssm))
			biGram_features1.append(bigram_features1(prob_pssm))
			biGram_features5.append(bigram_features5(prob_pssm))

		norm_pssm = prob_pssm + 0.01

		if(len(norm_pssm) < seqlen):
			for j in range(seqlen-len(norm_pssm)):
				norm_pssm = np.concatenate((norm_pssm,norm_pssm[0]*0))
		else:
			norm_pssm = norm_pssm[:seqlen]
		pssm.append(norm_pssm)

labels = np.array(labels)
# print("Labels=",labels)
num_classes =  len(np.unique(labels))
foldlabels = pd.get_dummies(labels).values
# print("foldlabels=",foldlabels)
sequences = np.array(seq)
# print(sequences.shape)
biGram0 = np.array(biGram_features0)
biGram1 = np.array(biGram_features1)
biGram2 = np.array(biGram_features2)
biGram3 = np.array(biGram_features3)
biGram4 = np.array(biGram_features4)
biGram5 = np.array(biGram_features5)
# np.random.shuffle(hmm)
# np.random.shuffle(pssm)
hmm = np.array(hmm)
pssm = np.array(pssm)

if(rawdata == 'hmm'):
	matrixdata = hmm
else:
	matrixdata = pssm

no_filters1 = 4

In [9]:
with tf.device('/device:GPU:0'):
	f=0
	config=tf.compat.v1.ConfigProto()
	config.gpu_options.allow_growth = True
	config.gpu_options.per_process_gpu_memory_fraction = 0.2
	tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))		

	acc_k_fold = []
	# kf = KFold(n_splits=10, shuffle=True, random_state=8)
	kf = StratifiedKFold(n_splits=10 shuffle=True, random_state=42)

	for train, test in kf.split(sequences, labels):
		f=f+1
		X_train, X_test = matrixdata[train], matrixdata[test]
		Y_train, Y_test = foldlabels[train], foldlabels[test]
		X_biGram0_Train, X_biGram0_Test = biGram0[train], biGram0[test]
		X_biGram1_Train, X_biGram1_Test = biGram1[train], biGram1[test]
		X_biGram2_Train, X_biGram2_Test = biGram2[train], biGram2[test]
		X_biGram3_Train, X_biGram3_Test = biGram3[train], biGram3[test]
		X_biGram4_Train, X_biGram4_Test = biGram4[train], biGram4[test]
		X_biGram5_Train, X_biGram5_Test = biGram5[train], biGram5[test]
		# X_Sgt_Train, X_Sgt_Test = all_sequences[train], all_sequences[test]

		# print("X_train:", X_train.shape)
		# print("Y_train:", Y_train.shape)
		# print("X_biGram0_Train:", X_biGram0_Train.shape)
		# print("X_biGram1_Train:", X_biGram1_Train.shape)
		# print("X_biGram2_Train:", X_biGram2_Train.shape)
		# print("X_biGram3_Train:", X_biGram3_Train.shape)
		# print("X_biGram4_Train:", X_biGram4_Train.shape)
		# print("X_biGram5_Train:", X_biGram5_Train.shape)

		cnn_input = Input(shape=(seqlen,20), name='cnn_input')		
		c_input = Reshape((seqlen,20,1))(cnn_input)		
		c_output1 = Conv2D(no_filters1, (5,5),  activation='tanh', strides=5, padding='same')(c_input)
		m_output1 = MaxPooling2D((3,3), strides=3, padding='same')(c_output1)
		f_input = Flatten()(m_output1)
		f_input = tf.expand_dims(f_input, axis=1)
		# print("Convolution Features Size: ", f_input.shape)
		bigram_input0 = Input(shape=(X_biGram0_Train.shape[1], X_biGram0_Train.shape[2]), name='bigram_input0')
		# bg0_input = Flatten()(bigram_input0)

		bigram_input1 = Input(shape=(X_biGram1_Train.shape[1], X_biGram1_Train.shape[2]), name='bigram_input1')
		# bg1_input = Flatten()(bigram_input1)

		bigram_input2 = Input(shape=(X_biGram2_Train.shape[1], X_biGram2_Train.shape[2]), name='bigram_input2')
		# bg2_input = Flatten()(bigram_input2)

		bigram_input3 = Input(shape=(X_biGram3_Train.shape[1], X_biGram3_Train.shape[2]), name='bigram_input3')
		# bg3_input = Flatten()(bigram_input3)

		bigram_input4 = Input(shape=(X_biGram4_Train.shape[1], X_biGram4_Train.shape[2]), name='bigram_input4')
		# bg4_input = Flatten()(bigram_input4)

		bigram_input5 = Input(shape=(X_biGram5_Train.shape[1], X_biGram5_Train.shape[2]), name='bigram_input5')
		# bg5_input = Flatten()(bigram_input5)

		bigram_input5 = Input(shape=(X_biGram5_Train.shape[1], X_biGram5_Train.shape[2]), name='bigram_input5')
		# bg5_input = Flatten()(bigram_input5)

		# sgt_input = Input(shape=(X_Sgt_Train.shape[1], X_Sgt_Train.shape[2]), name='sgt_input')
		# sgt_input_flatten = Flatten()(sgt_input)

		# hybrid_features = concatenate([bg0_input, bg1_input, f_input, bg2_input, bg3_input, bg4_input, bg5_input, sgt_input_flatten], axis=-1)
		# hybrid_features = bg0_input
		# hybrid_features = concatenate([bg0_input, bg1_input, bg2_input, bg3_input, bg4_input, bg5_input], axis=-1)
		# hybrid_features = concatenate([bigram_input0, bigram_input1, bigram_input2, bigram_input3, bigram_input4, bigram_input5], axis=2)
		# hybrid_features = bigram_input0
		hybrid_features = concatenate([f_input, bigram_input0, bigram_input1, bigram_input2, bigram_input3, bigram_input4, bigram_input5], axis=2)


		y_train_shape = [Y_train.shape[0], 1, Y_train.shape[1]]
		y_test_shape = [Y_test.shape[0], 1, Y_test.shape[1]]
		Y_train = tf.reshape(Y_train, y_train_shape)
		Y_test = tf.reshape(Y_test, y_test_shape)

		# print("Hybrid Features Size: ", hybrid_features.shape)
		l_output1 = layers.MultiHeadAttention(num_heads=2, key_dim=2)
		attention_out1 = l_output1(hybrid_features, hybrid_features)
		l_output1 = layers.Add()([attention_out1, hybrid_features])
		l_output1 = layers.LayerNormalization(epsilon=1e-6)(l_output1)
		d_output1 = Dense(512, activation='relu')(l_output1)
		d_output1 = Dense(2512, activation='relu')(d_output1)
		# l_input2 = layers.Add()([d_output1, l_output1])
		# l_input2 = layers.LayerNormalization(epsilon=1e-6)(l_input2)
		# l_output2 = layers.MultiHeadAttention(num_heads=2, key_dim=2)
		# attention_out2 = l_output2(l_input2, l_input2)
		# l_output2 = layers.Add()([attention_out2, l_input2])
		# l_output2 = layers.LayerNormalization(epsilon=1e-6)(l_output2)
		# d_output2 = Dense(512, activation='tanh')(l_output2)
		# d_output2 = Dense(2512, activation='tanh')(d_output2)
		# l_input3 = layers.Add()([d_output2, l_output2])
		# l_input3 = layers.LayerNormalization(epsilon=1e-6)(l_input3)
		# l_output3 = layers.MultiHeadAttention(num_heads=2, key_dim=2)
		# attention_out3 = l_output3(l_input3, l_input3)
		# l_output3 = layers.Add()([attention_out3, l_input3])
		# l_output3 = layers.LayerNormalization(epsilon=1e-6)(l_output3)
		# d_output3 = Dense(512, activation='tanh')(l_output3)
		# d_output3 = Dense(2512, activation='tanh')(d_output2)
		# l_input4 = layers.Add()([d_output3, l_output3])
		# l_input4 = layers.LayerNormalization(epsilon=1e-6)(l_input3)
		# l_output4 = layers.MultiHeadAttention(num_heads=2, key_dim=2)
		# attention_out4 = l_output4(l_input4, l_input4)
		# l_output4 = layers.Add()([attention_out4, l_input4])
		# l_output4 = layers.LayerNormalization(epsilon=1e-6)(l_output4)
		# d_output4 = Dense(512, activation='tanh')(l_output4)
		# d_output4 = Dense(2512, activation='tanh')(d_output4)
		# l_input5 = layers.Add()([l_output4, d_output4])
		# l_input5 = layers.LayerNormalization(epsilon=1e-6)(l_input5)
		# l_output5 = layers.MultiHeadAttention(num_heads=2, key_dim=2)
		# attention_out5 = l_output5(l_input5, l_input5)
		# l_output5 = layers.Add()([attention_out5, l_input5])
		# l_output5 = layers.LayerNormalization(epsilon=1e-6)(l_output5)
		# d_output5 = Dense(512, activation='tanh')(l_output5)
		# d_output5 = Dense(2512, activation='tanh')(d_output5)
		# l_input6 = layers.Add()([l_output5, d_output5])
		# l_input6 = layers.LayerNormalization(epsilon=1e-6)(l_input6)
		# l_output6 = layers.MultiHeadAttention(num_heads=2, key_dim=2)
		# attention_out6 = l_output6(l_input6, l_input6)
		# l_output6 = layers.Add()([attention_out6, l_input6])
		# l_output6 = layers.LayerNormalization(epsilon=1e-6)(l_output6)
		# d_output6 = Dense(512, activation='tanh')(l_output6)
		# d_output6 = Dense(2512, activation='tanh')(d_output6)
		# l_input7 = layers.Add()([l_output6, d_output6])
		# l_input7 = layers.LayerNormalization(epsilon=1e-6)(l_input7)
		d_output_last = Dense(512, activation='tanh')(d_output1)
		d_output_last = Dense(128, activation='tanh')(d_output_last)
		main_output = Dense(foldlabels.shape[1], activation='softmax', name='main_output', kernel_regularizer=l2(0.01))(d_output_last)
		model = Model(inputs=[cnn_input, bigram_input0, bigram_input1, bigram_input2, bigram_input3, bigram_input4, bigram_input5], outputs=[main_output])
		model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
		# model.summary()

		earlyStopping = EarlyStopping(monitor='val_accuracy', patience=50, verbose=0, mode='auto')
		load_file = "./model/"+dataset+"_SXG_BiGram_best.h5"
		checkpointer = ModelCheckpoint(monitor='val_accuracy', filepath=load_file, verbose=0, save_best_only=True)

		history=model.fit({'cnn_input': X_train, 'bigram_input0': X_biGram0_Train, 'bigram_input1': X_biGram1_Train, 'bigram_input2': X_biGram2_Train, 'bigram_input3': X_biGram3_Train, 'bigram_input4': X_biGram4_Train, 'bigram_input5': X_biGram5_Train}, {'main_output': Y_train}, 
			validation_data=({'cnn_input': X_test, 'bigram_input0': X_biGram0_Test, 'bigram_input1': X_biGram1_Test, 'bigram_input2': X_biGram2_Test, 'bigram_input3': X_biGram3_Test, 'bigram_input4': X_biGram4_Test, 'bigram_input5': X_biGram5_Test},{'main_output': Y_test}), 
			epochs=500, batch_size=64, callbacks=[checkpointer, earlyStopping], verbose=0)

		model.load_weights(load_file)
		score = model.evaluate({'cnn_input': X_test, 'bigram_input0': X_biGram0_Test, 'bigram_input1': X_biGram1_Test, 'bigram_input2': X_biGram2_Test, 'bigram_input3': X_biGram3_Test, 'bigram_input4': X_biGram4_Test, 'bigram_input5': X_biGram5_Test},{'main_output': Y_test}, verbose=0, batch_size=1)
		print("Fold-",f, ": ", score)

		acc_k_fold.append(score[1])

		pred_scores = model.predict({'cnn_input': X_test, 'bigram_input0': X_biGram0_Test, 'bigram_input1': X_biGram1_Test, 'bigram_input2': X_biGram2_Test, 'bigram_input3': X_biGram3_Test, 'bigram_input4': X_biGram4_Test, 'bigram_input5': X_biGram5_Test})
		print("pred_scores shape =", pred_scores.shape)

		# foldslabel=['Globin-like', 'Cytochrome C', 'DNA-binding 3-helical bundle', 'Immunoglobulin-like Beta-sandwich', 'Common fold of diphtheria toxin/trascription factors/transcription', 'Cupredoxins', 'TIM beta/alpha-barrel', 'NAD(P)-binding Rossmann-fold domains ', 'FAD/NAD(P)-binding domain', '4-Helical up-and-down bundle', '4-Helical cytokines', 'Aplha-EF-hand', 'SAM domain-like', 'Galactose-binding domain-like', 'Viral coat and capsid proteins', 'Concanavalin A-like lectins/glucanases', 'SH3-like barrel', 'OB-fold', 'Trefoil', 'Trypsin-like serine protease', 'Lipocalins', 'Double-stranded Beta-helix', 'Flavodoxin-like', 'Adenine nucleotide alpha hydrolase-like ', 'P-loop containing nucleoside triphosphate hydrolases ', 'Thioredoxin fold', 'Ribonuclease H-like motif', 'Phosphorylase/hydrolase-like', 'S-adenosyl-L-methionine-dependent methyltransferases', 'Alpha/beta-Hydrolases', 'Periplasmic binding protein-like I', 'Beta-Grasp (ubiquitin-like) ', 'Cystatin-like', 'Ferredoxin-like', 'Alpha-alpha super helix', 'Nucleoplasmin-like']

		# for i in range(pred_scores.shape[0]):
		# 	if np.sum(pred_scores[i,:]) != 0:
		# 		res=np.argmax(pred_scores[i,:])
		# 		print("Fold is:", foldslabel[res])

		# 	else:
		# 		print("Danger")

		# y_classes = y_prob.argmax(axis=-1)
		# print("Model Evaluate output", y_classes)


	resdata = "Class Features = "+str(clsfeature)+" -- 10-cross fold accuracy of Protein "+predtype+" Prediction of "+dataset+" using "+rawdata+" is :"+str(np.mean(acc_k_fold))+"\n"

	print(resdata)
	print("10 Fold Accuracies:", acc_k_fold)

print("# of Labels:", num_classes)
print("# of Labels:", foldlabels.shape[1])
print("hybrid_features count:", hybrid_features.shape)


Fold- 1 :  [1.637765645980835, 0.8464000225067139]
20/20 [==============================] - 1s 11ms/step
pred_scores shape = (625, 1, 171)
Fold- 2 :  [1.6080728769302368, 0.8464000225067139]
20/20 [==============================] - 0s 11ms/step
pred_scores shape = (625, 1, 171)
Fold- 3 :  [1.7202656269073486, 0.8399999737739563]
20/20 [==============================] - 0s 11ms/step
pred_scores shape = (625, 1, 171)
Fold- 4 :  [1.7978019714355469, 0.8159999847412109]
20/20 [==============================] - 0s 11ms/step
pred_scores shape = (625, 1, 171)
Fold- 5 :  [1.7183433771133423, 0.8223999738693237]
20/20 [==============================] - 1s 11ms/step
pred_scores shape = (625, 1, 171)
Fold- 6 :  [2.2258787155151367, 0.7247999906539917]
20/20 [==============================] - 0s 11ms/step
pred_scores shape = (625, 1, 171)
Fold- 7 :  [1.7343553304672241, 0.8288000226020813]
20/20 [==============================] - 0s 11ms/step
pred_scores shape = (625, 1, 171)
Fold- 8 :  [1.574793

Fold- 1 :  [1.1856778860092163, 0.8428571224212646]
3/3 [==============================] - 1s 33ms/step
pred_scores shape = (70, 1, 27)
Fold- 2 :  [0.7193385362625122, 0.8714285492897034]
3/3 [==============================] - 1s 40ms/step
pred_scores shape = (70, 1, 27)
Fold- 3 :  [0.7449996471405029, 0.9142857193946838]
3/3 [==============================] - 1s 32ms/step
pred_scores shape = (70, 1, 27)
Fold- 4 :  [1.0007456541061401, 0.8714285492897034]
3/3 [==============================] - 1s 25ms/step
pred_scores shape = (70, 1, 27)
Fold- 5 :  [1.1810381412506104, 0.8571428656578064]
3/3 [==============================] - 1s 43ms/step
pred_scores shape = (70, 1, 27)
Fold- 6 :  [0.9316855072975159, 0.8695651888847351]
3/3 [==============================] - 1s 28ms/step
pred_scores shape = (69, 1, 27)
Fold- 7 :  [1.3379615545272827, 0.8405796885490417]
3/3 [==============================] - 1s 23ms/step
pred_scores shape = (69, 1, 27)
Fold- 8 :  [0.9299620389938354, 0.8840579986572266]
3/3 [==============================] - 1s 33ms/step
pred_scores shape = (69, 1, 27)
Fold- 9 :  [1.2655516862869263, 0.8550724387168884]
3/3 [==============================] - 1s 35ms/step
pred_scores shape = (69, 1, 27)
Fold- 10 :  [0.6408186554908752, 0.8985507488250732]
3/3 [==============================] - 1s 26ms/step
pred_scores shape = (69, 1, 27)
Class Features = False -- 10-cross fold accuracy of Protein Fold Prediction of dd using hmm is :0.8704968869686127
